# Component mapper test script

## prepare ground truth mapping

In [1]:
from groundtruth_mapping import GroundTruthMapping

In [3]:
groundtruth_mapping = GroundTruthMapping()
groundtruth_mapping.load_mapping_from_json_file()
groundtruth_mapping.add_parent_component()
groundtruth_mapping.normalize_mapping()

In [4]:
#print groundtruth_mapping.complete_mappings

## get complete component lists 

In [5]:
from component_tree import ComponentTree

In [6]:
component_tree = ComponentTree()
component_tree.get_component_from_database()
component_tree.get_component_labels()

In [7]:
#print component_tree.component_defs
#print component_tree.component_labels

## tune model on training set

In [8]:
import train_model

/usr/local/google/home/ymzhang/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## tune model on training set 

In [9]:
model_metric = train_model.tune_model_on_trainset(groundtruth_mapping.complete_mappings,
                                                  component_tree.component_defs,
                                                  component_tree.component_labels,'decision_tree')

In [14]:
print "instance level accuracy:"
print model_metric["accuracy_instance"]
print "instance level recall:"
print model_metric["recall_instance"]
print "directory level recall"
print model_metric["recall_dir"]
#print model_metric["error_dir"]
#print model_metric["error_dir_missing_component"]

instance level accuracy:
0.999497467951
instance level recall:
0.918424987539
directory level recall
0.967988865692


## train model 

In [15]:
tfidf_transformer, bigram_vectorizer, components, clf = train_model.train_model(groundtruth_mapping.complete_mappings, 
                                                                                component_tree.component_defs, 
                                                                                component_tree.component_labels,  
                                                                                'decision_tree')

## check model on training set

In [27]:
model_metric_train = train_model.evaluate_model_on_trainset(groundtruth_mapping.complete_mappings, 
                                                      component_tree.component_labels, 
                                                      bigram_vectorizer, tfidf_transformer, clf)

In [28]:
print model_metric_train["accuracy_instance"]
print model_metric_train["recall_instance"]
print model_metric_train["recall_dir"]
#print model_metric_train["error_dir"]
#print model_metric_train["error_dir_missing_component"]

1.0
1.0
1.0
defaultdict(<type 'list'>, {})
defaultdict(<type 'list'>, {})


## test prediction with a list of directories 

In [25]:
check_list = [ 
    "src/third_party/WebKit/Source/modules/geolocation", 
    "src/third_party/WebKit/Source/modules/vibration", 
    "src/components/gcm_driver/android", 
    "src/content/public/android/javatests/src/org/chromium/content/browser", 
    "src/device/geolocation/public/interfaces"
]

In [26]:
predict_compos = train_model.predict_component(check_list, tfidf_transformer, bigram_vectorizer, components, clf)
for i in range(0, len(check_list)):
    print ", ".join(predict_compos[i])

Blink, Blink>Location
Blink, Blink>Storage
Services, Services>CloudMessaging
Blink, Blink>Storage
Manifest
